In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

In [4]:
data = pd.read_csv('data.csv')
top16 = data.nlargest(16, 'MEDV').index
data.drop(top16, inplace=True)
data.RM = data.RM.astype(int)

In [5]:
categorias = []

for i in data.RM.iteritems():

    valor = (i[1])
    if valor <=4:
        categorias.append('Pequeno')
    elif valor < 7:
        categorias.append('Médio')
    else:
        categorias.append('Grande')

data['categorias'] = categorias

In [6]:
data.head()

,Unnamed: 0,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,categorias
0,0,0.00632,18.0,2.31,0.0,0.538,6,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,Médio
1,1,0.02731,0.0,7.07,0.0,0.469,6,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,Médio
2,2,0.02729,0.0,7.07,0.0,0.469,7,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,Grande
3,3,0.03237,0.0,2.18,0.0,0.458,6,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,Médio
4,4,0.06905,0.0,2.18,0.0,0.458,7,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,Grande
